In [84]:
#Trazendo os dados de vendas
import pandas as pd

sales = pd.read_csv('bases/data_base.csv')
print(sales)

            date                 datetime cash_type                 card  \
0     2024-03-01  2024-03-01 10:15:50.520      card  ANON-0000-0000-0001   
1     2024-03-01  2024-03-01 12:19:22.539      card  ANON-0000-0000-0002   
2     2024-03-01  2024-03-01 12:20:18.089      card  ANON-0000-0000-0002   
3     2024-03-01  2024-03-01 13:46:33.006      card  ANON-0000-0000-0003   
4     2024-03-01  2024-03-01 13:48:14.626      card  ANON-0000-0000-0004   
...          ...                      ...       ...                  ...   
3631  2025-03-23  2025-03-23 10:34:54.894      card  ANON-0000-0000-1158   
3632  2025-03-23  2025-03-23 14:43:37.362      card  ANON-0000-0000-1315   
3633  2025-03-23  2025-03-23 14:44:16.864      card  ANON-0000-0000-1315   
3634  2025-03-23  2025-03-23 15:47:28.723      card  ANON-0000-0000-1316   
3635  2025-03-23  2025-03-23 18:11:38.635      card  ANON-0000-0000-1275   

      money    coffee_name  
0     38.70          Latte  
1     38.70  Hot Chocolate  


In [82]:
#Filtrando os dados necessários

sales = sales[["date", "coffee_name"]]
sales = sales.groupby("date").size().reset_index(name="qtd")
# sales.set_index("date", inplace=True) 
# sales["coffee_name"] = sales["coffee_name"].count()

display(sales)

,date,qtd
0,2024-03-01,5
1,2024-03-02,3
2,2024-03-03,6
3,2024-03-04,3
4,2024-03-05,4
...,...,...
376,2025-03-19,8
377,2025-03-20,8
378,2025-03-21,5
379,2025-03-22,6


In [85]:
sales = sales[["date", "coffee_name"]]
sales = sales.groupby("date")["coffee_name"].count()

display(sales)

date
2024-03-01    11
2024-03-02     7
2024-03-03    10
2024-03-04     4
2024-03-05     9
              ..
2025-03-19    21
2025-03-20    20
2025-03-21    20
2025-03-22    12
2025-03-23     6
Name: coffee_name, Length: 381, dtype: int64